In [1]:
from google.colab import drive
drive.mount('/content/drive')
import os
os.chdir('/content/drive/MyDrive/CS5246/AITA')

Mounted at /content/drive


In [2]:
!pip install contractions

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 289.9/289.9 kB 5.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 110.7/110.7 kB 12.9 MB/s eta 0:00:00


In [3]:
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
import seaborn as sns
import re
import string
import contractions

import nltk
nltk.download('punkt')
nltk.download('wordnet')
nltk.download('stopwords')
from nltk.tokenize import sent_tokenize, word_tokenize
from nltk.stem.porter import PorterStemmer
from nltk.stem.snowball import SnowballStemmer
from nltk.stem.lancaster import LancasterStemmer
from nltk.stem import WordNetLemmatizer
from nltk.corpus import stopwords
from nltk.cluster.util import cosine_distance
import networkx as nx

from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.model_selection import cross_val_score, GridSearchCV, train_test_split
from sklearn.naive_bayes import MultinomialNB
from sklearn.linear_model import LogisticRegression
from sklearn import metrics
from sklearn.metrics import f1_score

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


In [4]:
def read_article(text):
    # Tokenize the text into sentences
    sentences = sent_tokenize(text)
    return len(sentences)

porter_stemmer = PorterStemmer()
wordnet_lemmatizer = WordNetLemmatizer()

def tokenize_and_stem(text):
    expanded_text = contractions.fix(text)
    clean_text = re.sub(r'http[s]?://\S+', '', expanded_text)
    tokens = word_tokenize(clean_text)
    lemmas = [wordnet_lemmatizer.lemmatize(token) for token in tokens]
    return lemmas

### Load data

In [5]:
train_part1 = pd.read_csv('train_with_pagerank_part1.csv')
train_part2 = pd.read_csv('train_with_pagerank_part2.csv')
train = pd.concat([train_part1, train_part2], axis=0)
test = pd.read_csv('test_with_pagerank.csv')
df = pd.concat([train, test], axis=0)

In [ ]:
df['sent_length'] = df['content'].apply(read_article)
df['token_length'] = df['content'].apply(lambda x: len(tokenize_and_stem(x)))
df.describe()

,flag,sent_length,token_length
count,16872.000000,16872.000000,16872.000000
mean,0.399360,21.756520,466.342817
std,0.489781,11.416171,217.605811
min,0.000000,1.000000,9.000000
25%,0.000000,14.000000,309.000000
50%,0.000000,20.000000,438.000000
75%,1.000000,27.000000,603.000000
max,1.000000,194.000000,2714.000000


In [ ]:
df.query('token_length<50')

,flag,content,sent1,sent2,sent3,sent4,sent5,pagerank,sent_length,token_length
100,1,AITA for refusing to apologize to my parents? | F,AITA for refusing to apologize to my parents?,| F,NaN,NaN,NaN,AITA for refusing to apologize to my parents?| F,2,11
530,1,AITA because I made a new female employee cry ...,AITA because I made a new female employee cry ...,| .,NaN,NaN,NaN,AITA because I made a new female employee cry ...,2,18
539,1,AITA for reminding my brother that he’s adopte...,AITA for reminding my brother that he is adopt...,| FUCK ALL OF YOU,NaN,NaN,NaN,AITA for reminding my brother that he is adopt...,2,23
561,1,AITA for not always putting kitchen utensils i...,AITA for not always putting kitchen utensils i...,| My husband just exploded on me when he could...,"He does all the cooking, I do all the cleaning.",NaN,NaN,AITA for not always putting kitchen utensils i...,3,46
5126,1,WIBTA if I dropped my kids with their dad and ...,WIBTA if I dropped my kids with their dad and ...,| IATA.,Thanks.,Good night,NaN,WIBTA if I dropped my kids with their dad and ...,4,22
5133,1,AITA for getting angry at kid (and her mom lat...,AITA for getting angry at kid (and her mom lat...,| CLOSE,NaN,NaN,NaN,AITA for getting angry at kid (and her mom lat...,2,22
5259,1,AITA: I got a replica necklace for a friend th...,AITA: I got a replica necklace for a friend th...,NaN,NaN,NaN,NaN,AITA: I got a replica necklace for a friend th...,1,17
5326,1,AITA for calling my dad a charity case and a g...,AITA for calling my dad a charity case and a g...,| .,NaN,NaN,NaN,AITA for calling my dad a charity case and a g...,2,25
5420,1,AITA for effectively banning my nephew from ma...,AITA for effectively banning my nephew from ma...,| I am not good at describing noises.,JFC,NaN,NaN,AITA for effectively banning my nephew from ma...,3,19
5440,1,AITA for not going to jump my fiance's car? | .,AITA for not going to jump my fiance's car?,| .,NaN,NaN,NaN,AITA for not going to jump my fiance's car? | ...,2,13


#Topic Modelling (to find what topics are in the reddit post)

In [ ]:
#topic modelling
import gensim
from gensim import corpora
from gensim.models import LdaModel

# Tokenize and preprocess the text data
stop_words = set(stopwords.words('english'))
punctuations = set(string.punctuation)
punctuations.union(set(['\'', '\`', '\“']))

def preprocess_text(text):
    expanded_text = contractions.fix(text) # adjust contractions
    clean_text = re.sub(r'http[s]?://\S+', '', expanded_text)
    tokens = word_tokenize(clean_text.lower())  # Tokenize text and convert to lowercase
    tokens = [token for token in tokens if token not in stop_words and token not in punctuations]  # Remove stopwords and punctuation
    return tokens

preprocess_text = df['content'].apply(preprocess_text)

# Create dictionary and corpus for LDA model
dictionary = corpora.Dictionary(preprocess_text)
corpus = [dictionary.doc2bow(post) for post in preprocess_text]

# Train LDA model
lda_model = LdaModel(corpus, num_topics=5, id2word=dictionary, passes=10)

# Print topics
for idx, topic in lda_model.print_topics(-1):
    print(f"Topic {idx}: {topic}")

Topic 0: 0.015*"dog" + 0.010*"would" + 0.009*"go" + 0.009*"trip" + 0.008*"work" + 0.008*"one" + 0.007*"people" + 0.007*"class" + 0.007*"school" + 0.006*"us"
Topic 1: 0.013*"like" + 0.012*"said" + 0.010*"would" + 0.009*"told" + 0.008*"“" + 0.008*"”" + 0.008*"really" + 0.008*"know" + 0.008*"friend" + 0.007*"friends"
Topic 2: 0.018*"''" + 0.018*"``" + 0.012*"food" + 0.010*"said" + 0.009*"like" + 0.008*"eat" + 0.006*"get" + 0.006*"dinner" + 0.006*"one" + 0.006*"told"
Topic 3: 0.012*"would" + 0.012*"money" + 0.010*"work" + 0.009*"house" + 0.009*"get" + 0.009*"pay" + 0.008*"time" + 0.007*"home" + 0.006*"job" + 0.006*"back"
Topic 4: 0.012*"family" + 0.012*"would" + 0.011*"told" + 0.011*"mom" + 0.010*"said" + 0.009*"sister" + 0.008*"dad" + 0.008*"'s" + 0.008*"wife" + 0.008*"daughter"


#2 part logistic regression

###Preprocessing

In [ ]:
def flatten(data):
    results = []
    for index, row in data.iterrows():
      results.extend(row)
    return results

In [ ]:
#flatten the data so that each sentence is now 1 data point
X_train_flattened = flatten(train[['sent1', 'sent2', 'sent3', 'sent4', 'sent5']].fillna(''))
X_test_flattened = flatten(test[['sent1', 'sent2', 'sent3', 'sent4', 'sent5']].fillna(''))

In [ ]:
#Vectorise the sentences
vectorizer = TfidfVectorizer(tokenizer=tokenize_and_stem, token_pattern=None)

vectorizer.fit(X_train_flattened)
X_train_vector = vectorizer.transform(X_train_flattened)
X_test_vector = vectorizer.transform(X_test_flattened)

In [ ]:
#Repeat each label 5 times to match the new flattened data
y_train_flattened = np.repeat(train['flag'], 5)
y_test_flattened = np.repeat(test['flag'], 5)

### Logistic Regression

In [ ]:
#Stage 1 Logistic Regression, to train the output for each sentence
stage_1 = LogisticRegression(solver="saga")
stage_1.fit(X_train_vector, y_train_flattened)
X_train_intermediate_results = stage_1.predict(X_train_vector)
X_test_intermediate_results = stage_1.predict(X_test_vector)

In [ ]:
#reshape to push into stage 2 logistic regression
X_train_intermediate_input = X_train_intermediate_results.reshape(int(len(X_train_intermediate_results)/5), 5)
X_test_intermediate_input = X_test_intermediate_results.reshape(int(len(X_test_intermediate_results)/5), 5)
print(X_train_intermediate_input.shape)
print(X_test_intermediate_input.shape)

(10872, 5)
(6000, 5)


In [ ]:
#Stage 2 Logistic Regression training score
cross_val_scores = cross_val_score(LogisticRegression(solver="saga"), X_train_intermediate_input, train['flag'], cv=10, scoring="f1")
print(f"cross-validation f1 scores:", cross_val_scores)
print(f"average cross-validation f1 score:", cross_val_scores.mean())

cross-validation f1 scores: [0.86314022 0.86317568 0.85311203 0.86248983 0.8349835  0.82246078
 0.80370682 0.74159292 0.72128228 0.73777778]
average cross-validation f1 score: 0.8103721832537669


In [ ]:
#Stage 2 Logistic Regression training score
cross_val_scores = cross_val_score(LogisticRegression(solver="saga"), X_train_intermediate_input, train['flag'], cv=10, scoring="accuracy")
print(f"cross-validation accuracy:", cross_val_scores)
print(f"average cross-validation accuracy:", cross_val_scores.mean())

cross-validation accuracy: [0.85018382 0.85110294 0.83716651 0.84452622 0.81600736 0.80220791
 0.78564857 0.73137075 0.71205152 0.72861086]
average cross-validation accuracy: 0.7958876461118025


In [ ]:
#Stage 2 Logistic Regression testing score
stage_2 = LogisticRegression(solver="saga")
stage_2.fit(X_train_intermediate_input, train['flag'])
y_pred = stage_2.predict(X_test_intermediate_input)
print(f"accuracy:", metrics.accuracy_score(test['flag'], y_pred))
print(f"f1-score:", metrics.f1_score(test['flag'], y_pred))

accuracy: 0.6048333333333333
f1-score: 0.3915832691814216


In [ ]:
#Check whether majority polling gives the same results
y_test = [round(np.mean(x)) for x in X_test_intermediate_input]
print(f"accuracy:", metrics.accuracy_score(test['flag'], y_test))
print(f"f1-score:", metrics.f1_score(test['flag'], y_test ))

accuracy: 0.6048333333333333
f1-score: 0.3915832691814216


### NBaive Bayes

In [ ]:
#Stage 1 NB, to train the output for each sentence
stage_1 = MultinomialNB()
stage_1.fit(X_train_vector, y_train_flattened)
X_train_intermediate_results = stage_1.predict(X_train_vector)
X_test_intermediate_results = stage_1.predict(X_test_vector)

In [ ]:
#reshape to push into stage 2 NB
X_train_intermediate_input = X_train_intermediate_results.reshape(int(len(X_train_intermediate_results)/5), 5)
X_test_intermediate_input = X_test_intermediate_results.reshape(int(len(X_test_intermediate_results)/5), 5)
print(X_train_intermediate_input.shape)
print(X_test_intermediate_input.shape)

(10872, 5)
(6000, 5)


In [ ]:
#Stage 2 NB training score
cross_val_scores = cross_val_score(MultinomialNB(), X_train_intermediate_input, train['flag'], cv=10, scoring="f1")
print(f"cross-validation f1 scores:", cross_val_scores)
print(f"average cross-validation f1 score:", cross_val_scores.mean())

cross-validation f1 scores: [0.69073406 0.68848485 0.68647665 0.69036145 0.68856448 0.68898357
 0.68253968 0.68515096 0.69114991 0.67617866]
average cross-validation f1 score: 0.6868624252975506


In [ ]:
#Stage 2 NB training score
cross_val_scores = cross_val_score(MultinomialNB(), X_train_intermediate_input, train['flag'], cv=10, scoring="accuracy")
print(f"cross-validation accuracy:", cross_val_scores)
print(f"average cross-validation accuracy:", cross_val_scores.mean())

cross-validation accuracy: [0.52757353 0.52757353 0.52437902 0.52713891 0.52897884 0.5298988
 0.52161914 0.5298988  0.52805888 0.51977921]
average cross-validation accuracy: 0.5264898668759133


In [ ]:
#Stage 2 NB testing score
stage_2 = MultinomialNB()
stage_2.fit(X_train_intermediate_input, train['flag'])
y_pred = stage_2.predict(X_test_intermediate_input)
print(f"accuracy:", metrics.accuracy_score(test['flag'], y_pred))
print(f"f1-score:", metrics.f1_score(test['flag'], y_pred))

accuracy: 0.18566666666666667
f1-score: 0.2854635858438139


In [ ]:
#Check whether majority polling gives the same results
y_test = [round(np.mean(x)) for x in X_test_intermediate_input]
print(f"accuracy:", metrics.accuracy_score(test['flag'], y_test))
print(f"f1-score:", metrics.f1_score(test['flag'], y_test ))

accuracy: 0.5078333333333334
f1-score: 0.3595749295163739


### Mixed

In [ ]:
#Stage 1 Logistic Regression, to train the output for each sentence
stage_1 = MultinomialNB()
stage_1.fit(X_train_vector, y_train_flattened)
X_train_intermediate_results = stage_1.predict(X_train_vector)
X_test_intermediate_results = stage_1.predict(X_test_vector)

In [ ]:
#reshape to push into stage 2 logistic regression
X_train_intermediate_input = X_train_intermediate_results.reshape(int(len(X_train_intermediate_results)/5), 5)
X_test_intermediate_input = X_test_intermediate_results.reshape(int(len(X_test_intermediate_results)/5), 5)
print(X_train_intermediate_input.shape)
print(X_test_intermediate_input.shape)

(10872, 5)
(6000, 5)


In [ ]:
#Stage 2 Logistic Regression training score
cross_val_scores = cross_val_score(LogisticRegression(solver="saga"), X_train_intermediate_input, train['flag'], cv=10, scoring="f1")
print(f"cross-validation f1 scores:", cross_val_scores)
print(f"average cross-validation f1 score:", cross_val_scores.mean())

cross-validation scores: [0.87012987 0.88474026 0.86031746 0.87066246 0.85205267 0.83081811
 0.8058489  0.78613694 0.74223035 0.76678445]
average cross-validation score: 0.8269721475672949


In [ ]:
#Stage 2 Logistic Regression training score
cross_val_scores = cross_val_score(LogisticRegression(solver="saga"), X_train_intermediate_input, train['flag'], cv=10, scoring="accuracy")
print(f"cross-validation accuracy:", cross_val_scores)
print(f"average cross-validation accuracy:", cross_val_scores.mean())

cross-validation accuracy: [0.85294118 0.86948529 0.83808648 0.84912603 0.82428703 0.80404784
 0.78012879 0.76724931 0.74057038 0.75712971]
average cross-validation accuracy: 0.8083052045565235


In [ ]:
#Stage 2 Logistic Regression testing score
stage_2 = LogisticRegression(solver="saga")
stage_2.fit(X_train_intermediate_input, train['flag'])
y_pred = stage_2.predict(X_test_intermediate_input)
print(f"accuracy:", metrics.accuracy_score(test['flag'], y_pred))
print(f"f1-score:", metrics.f1_score(test['flag'], y_pred))

accuracy: 0.5078333333333334
f1-score: 0.3595749295163739


In [ ]:
#Check whether majority polling gives the same results
y_test = [round(np.mean(x)) for x in X_test_intermediate_input]
print(f"accuracy:", metrics.accuracy_score(test['flag'], y_test))
print(f"f1-score:", metrics.f1_score(test['flag'], y_test ))

accuracy: 0.5078333333333334
f1-score: 0.3595749295163739


#Justice model

In [6]:
justice = pd.read_csv('/content/drive/MyDrive/CS5246/AITA/justice_test.csv')
justice.head()

,label,scenario
0,1,I used to walk around the park with my coworke...
1,0,I used to walk around the park with my coworke...
2,1,I used to walk around the park with my coworke...
3,0,I used to walk around the park with my coworke...
4,0,I usually give a gift to my girlfriend on Vale...


In [7]:
def flatten(data):
    results = []
    for index, row in data.iterrows():
      results.extend(row)
    return results

In [11]:
#Vectorise the sentences
vectorizer2 = TfidfVectorizer(tokenizer=tokenize_and_stem, token_pattern=None)

vectorizer2.fit(justice['scenario'])
justice_train_vector = vectorizer2.transform(justice['scenario'])

In [8]:
#flatten the data so that each sentence is now 1 data point
X_train_flattened = flatten(train[['sent1', 'sent2', 'sent3', 'sent4', 'sent5']].fillna(''))
X_test_flattened = flatten(test[['sent1', 'sent2', 'sent3', 'sent4', 'sent5']].fillna(''))

In [15]:
#Vectorise the sentences
X_train_vector = vectorizer2.transform(X_train_flattened)
X_test_vector = vectorizer2.transform(X_test_flattened)

In [10]:
#Repeat each label 5 times to match the new flattened data
y_train_flattened = np.repeat(train['flag'], 5)
y_test_flattened = np.repeat(test['flag'], 5)

### Logistic Regression with Justice

In [12]:
#Stage 1 Logistic Regression training score
cross_val_scores = cross_val_score(LogisticRegression(solver="saga"), justice_train_vector, justice['label'], cv=10, scoring="f1")
print(f"cross-validation f1 scores:", cross_val_scores)
print(f"average cross-validation f1 score:", cross_val_scores.mean())

cross-validation f1 scores: [0.592      0.63736264 0.5859375  0.58518519 0.57471264 0.60498221
 0.61371841 0.57692308 0.56716418 0.55686275]
average cross-validation f1 score: 0.5894848585309618


In [13]:
#Stage 1 Logistic Regression training score
cross_val_scores = cross_val_score(LogisticRegression(solver="saga"), justice_train_vector, justice['label'], cv=10, scoring="accuracy")
print(f"cross-validation accuracy:", cross_val_scores)
print(f"average cross-validation accuracy:", cross_val_scores.mean())

cross-validation accuracy: [0.6199262  0.63468635 0.60516605 0.58671587 0.58518519 0.58888889
 0.6037037  0.58888889 0.57037037 0.58148148]
average cross-validation accuracy: 0.5965012983463168


In [16]:
#Stage 1 Logistic Regression, to train the output for each sentence
stage_1 = LogisticRegression(solver="saga")
stage_1.fit(justice_train_vector, justice['label'])
X_train_intermediate_results = stage_1.predict(X_train_vector)
X_test_intermediate_results = stage_1.predict(X_test_vector)

In [17]:
#reshape to push into stage 2 logistic regression
X_train_intermediate_input = X_train_intermediate_results.reshape(int(len(X_train_intermediate_results)/5), 5)
X_test_intermediate_input = X_test_intermediate_results.reshape(int(len(X_test_intermediate_results)/5), 5)
print(X_train_intermediate_input.shape)
print(X_test_intermediate_input.shape)

(10872, 5)
(6000, 5)


In [18]:
#Stage 2 Logistic Regression training score
cross_val_scores = cross_val_score(LogisticRegression(solver="saga"), X_train_intermediate_input, train['flag'], cv=10, scoring="f1")
print(f"cross-validation f1 scores:", cross_val_scores)
print(f"average cross-validation f1 score:", cross_val_scores.mean())

cross-validation f1 scores: [0.62271644 0.61487603 0.63234111 0.59331703 0.59347653 0.57491857
 0.55867769 0.54483348 0.56066946 0.55829228]
average cross-validation f1 score: 0.5854118616618903


In [20]:
#Stage 2 Logistic Regression training score
cross_val_scores = cross_val_score(LogisticRegression(solver="saga"), X_train_intermediate_input, train['flag'], cv=10, scoring="accuracy")
print(f"cross-validation accuracy:", cross_val_scores)
print(f"average cross-validation accuracy:", cross_val_scores.mean())

cross-validation accuracy: [0.56341912 0.57169118 0.57957682 0.54093836 0.5298988  0.51977921
 0.50873965 0.50965961 0.51701932 0.5050598 ]
average cross-validation accuracy: 0.5345781867254722


In [21]:
#Stage 2 Logistic Regression testing score
stage_2 = LogisticRegression(solver="saga")
stage_2.fit(X_train_intermediate_input, train['flag'])
y_pred = stage_2.predict(X_test_intermediate_input)
print(f"accuracy:", metrics.accuracy_score(test['flag'], y_pred))
print(f"f1-score:", metrics.f1_score(test['flag'], y_pred))

accuracy: 0.4861666666666667
f1-score: 0.3026464600769057


In [22]:
#Check whether majority polling gives the same results
y_test = [round(np.mean(x)) for x in X_test_intermediate_input]
print(f"accuracy:", metrics.accuracy_score(test['flag'], y_test))
print(f"f1-score:", metrics.f1_score(test['flag'], y_test ))

accuracy: 0.204
f1-score: 0.2643253234750462


### NBaive Bayes

In [23]:
#Stage 1 Logistic Regression training score
cross_val_scores = cross_val_score(MultinomialNB(), justice_train_vector, justice['label'], cv=10, scoring="f1")
print(f"cross-validation f1 scores:", cross_val_scores)
print(f"average cross-validation f1 score:", cross_val_scores.mean())

cross-validation f1 scores: [0.64028777 0.62222222 0.64028777 0.55677656 0.59859155 0.61333333
 0.6271777  0.58608059 0.52985075 0.61484099]
average cross-validation f1 score: 0.6029449223293202


In [24]:
#Stage 1 Logistic Regression training score
cross_val_scores = cross_val_score(MultinomialNB(), justice_train_vector, justice['label'], cv=10, scoring="accuracy")
print(f"cross-validation accuracy:", cross_val_scores)
print(f"average cross-validation accuracy:", cross_val_scores.mean())

cross-validation accuracy: [0.63099631 0.62361624 0.63099631 0.55350554 0.57777778 0.57037037
 0.6037037  0.58148148 0.53333333 0.5962963 ]
average cross-validation accuracy: 0.5902077354106874


In [26]:
#Stage 1 Logistic Regression, to train the output for each sentence
stage_1 = MultinomialNB()
stage_1.fit(justice_train_vector, justice['label'])
X_train_intermediate_results = stage_1.predict(X_train_vector)
X_test_intermediate_results = stage_1.predict(X_test_vector)

In [27]:
#reshape to push into stage 2 logistic regression
X_train_intermediate_input = X_train_intermediate_results.reshape(int(len(X_train_intermediate_results)/5), 5)
X_test_intermediate_input = X_test_intermediate_results.reshape(int(len(X_test_intermediate_results)/5), 5)
print(X_train_intermediate_input.shape)
print(X_test_intermediate_input.shape)

(10872, 5)
(6000, 5)


In [29]:
#Stage 2 NB training score
cross_val_scores = cross_val_score(MultinomialNB(), X_train_intermediate_input, train['flag'], cv=10, scoring="f1")
print(f"cross-validation f1 scores:", cross_val_scores)
print(f"average cross-validation f1 score:", cross_val_scores.mean())

cross-validation f1 scores: [0.69073406 0.69073406 0.69036145 0.69036145 0.69114991 0.69114991
 0.69114991 0.69114991 0.69114991 0.69114991]
average cross-validation f1 score: 0.690909046043399


In [30]:
#Stage 2 NB training score
cross_val_scores = cross_val_score(MultinomialNB(), X_train_intermediate_input, train['flag'], cv=10, scoring="accuracy")
print(f"cross-validation accuracy:", cross_val_scores)
print(f"average cross-validation accuracy:", cross_val_scores.mean())

cross-validation accuracy: [0.52757353 0.52757353 0.52713891 0.52713891 0.52805888 0.52805888
 0.52805888 0.52805888 0.52805888 0.52805888]
average cross-validation accuracy: 0.527777815357974


In [31]:
#Stage 2 NB testing score
stage_2 = MultinomialNB()
stage_2.fit(X_train_intermediate_input, train['flag'])
y_pred = stage_2.predict(X_test_intermediate_input)
print(f"accuracy:", metrics.accuracy_score(test['flag'], y_pred))
print(f"f1-score:", metrics.f1_score(test['flag'], y_pred))

accuracy: 0.16666666666666666
f1-score: 0.2857142857142857


In [32]:
#Check whether majority polling gives the same results
y_test = [round(np.mean(x)) for x in X_test_intermediate_input]
print(f"accuracy:", metrics.accuracy_score(test['flag'], y_test))
print(f"f1-score:", metrics.f1_score(test['flag'], y_test ))

accuracy: 0.26466666666666666
f1-score: 0.2582380632145259
